In [40]:
from pwn import *
from jvdsn.attacks.knapsack.low_density import attack

In [49]:
def to_bits(m):
    _bin = lambda b: [1 if b & (1 << n) else 0 for n in range(7)]
    return sum([_bin(b) for b in m], [])

def to_bytes(bits):
    _byte = lambda b: sum([b[i] << i for i in range(7)])
    return bytes([_byte(bits[i : i + 7]) for i in range(0, len(bits), 7)])

with open("./output.txt", "r") as f:
    exec(f.readline().strip())
    p_msg = f.read().encode()

In [46]:
def recover_pubkey(enc):
    msg_bits = to_bits(p_msg)[:]
    n = len(msg_bits)
    
    res = []
    info("Start...")
    for keysize in range(1, 92):
        b = vector(ZZ, enc[:n//keysize])
        A = []
        for i in range(n//keysize):
            A.append(msg_bits[i*keysize : (i+1)*keysize])
        A = matrix(n//keysize, keysize, A)
        
        try:
            x = A.solve_right(b)
            if all(xi > 0 for xi in x):
                info("Found!")
                print("\rpubkey:", x)
                print("\rkeysize:", keysize)
                res.append([int(xi) for xi in x])
        except:
            pass
    info("Done!")
    return res

pubkey = recover_pubkey(enc)[0]

[*] Start...
[*] Found!
pubkey: (202268999244591785584566464753289, 429481650350982224726540148422072, 513970026512572516113525776325248, 28117338492482390640953333703346, 186478220980676412258710097278824, 462107941653161013119183689513668, 251033680402139409627861954081627, 387816645824497842897510344498537, 164216600494137571823429526298731, 71778558816306530276260088958157, 479183011393572102154187215734921, 188237385509602029366682887942757, 164510164493942220113929554143015, 351957897387419262141620439154413, 559564023232732499808238313579215, 132277552177561352557338493209814, 178940764650522512349239779345091, 18866483643001119992915813508719, 65129268506159449846196687577134, 167844715331905178809461440317639, 187057779217236112720777337188447, 255235457949798322226331129469322, 542112960942493765359209270647723, 114204524812950672204089546427243, 53252437125337397373227709083049, 247770731648992733096183772815536, 370278558235654291454711551198693, 204891318167558074435123111

In [50]:
message = b""

for c in enc:
    try:
        bits = attack(pubkey, c)
        message += to_bytes(bits)
    except:
        message += b"?"*10

print(message)

b"The Merkle-Hellman Knapsack Cryptosystem, developed by Ralph Merkle and Martin Hellman, is a public-key encryption algorithm known for its resistance to attacks using conventional computers. It operates on the principle of the knapsack problem, making it difficult to solve without the private key.\nIn this cry?????????? a superincreasing knapsack is created as the public key. Each element of the knapsack is generated using a specific algorithm, ensuring that the sum of any subset of elements is unique. This property makes it challenging to deduce the original combination used to create the knapsack.\nTo e??????????essage, the plaintext is divided into binary bits and combined with the public key. This process results in a ciphertext that obscures the original message. Decrypting the ciphertext requires the knowledge of the private key, which is a set of carefully selected parameters used to generate the knapsack.\nThe security of the Merkle-Hellman Knapsack Cryptosystem relies on the